In [2]:
from processors import *
import warnings, inspect
from coffea import processor
from coffea.nanoevents.methods import vector, candidate
from coffea.nanoevents import BaseSchema
#warnings.filterwarnings("ignore")

In [4]:
# prefix = '/eos/uscms/store/user/christiw/bparking/V1p19/MC_Fall18/v1/v9/normalized/'

# filenames = [
#     'BToKPhi_MuonGenFilter_PhiToPi0Pi0_mPhi0p3_ctau1000_1pb_weighted.root',
#     'BToKPhi_MuonGenFilter_PhiToPi0Pi0_mPhi0p3_ctau300_1pb_weighted.root',
#     'BToKPhi_MuonGenFilter_PhiToPiPlusPiMinus_mPhi0p3_ctau1000_1pb_weighted.root',
#     'BToKPhi_MuonGenFilter_PhiToPiPlusPiMinus_mPhi0p3_ctau300_1pb_weighted.root',
# ]

# fileset = {
#     f"{file.split('_')[2]}_{file.split('_')[3]}_{file.split('_')[4]}": 
#     [prefix+file] 
#     for file in filenames
# }

# fileset['background'] = ['/eos/uscms/store/user/christiw/bparking/V1p19/Data2018_UL/v9/normalized/ParkingBPH4_2018A_goodLumi.root']

#=======================================================================

prefix = 'root://cmseos.fnal.gov//store/user/ahayrape/BigNtupler/'

fileset = {
            'PhiToPi0Pi0_mPhi0p3_ctau300':
                [prefix + 'PhiToPi0Pi0_mPhi0p3_ctau300.root'],

            'PhiToPi0Pi0_mPhi1p0_ctau1000':
                [prefix + 'PhiToPi0Pi0_mPhi1p0_ctau1000.root'],

            'PhiToPiPlusPiMinus_mPhi0p3_ctau300':
                [prefix + 'PhiToPiPlusPiMinus_mPhi0p3_ctau300.root'],

            'PhiToPiPlusPiMinus_mPhi1p0_ctau1000':
                [prefix + 'PhiToPiPlusPiMinus_mPhi1p0_ctau1000.root'],

            'PhiToPiPlusPiMinus_mPhi1p0_ctau300':
                [prefix + 'PhiToPiPlusPiMinus_mPhi1p0_ctau300.root'],
         }

# fileset['background'] = [prefix + 'Background.root']
fileset['background'] = ['/eos/uscms/store/user/christiw/bparking/V1p19/Data2018_UL/v9/normalized/ParkingBPH4_2018A_goodLumi.root']


In [18]:
processors = {
    # 'ProcessorV1': ProcessorV1(),
    'ProcessorV2': ProcessorV2(),
    #'ProcessorV3': ProcessorV3('CSC'),
}


test_mode = False

for proc in processors:
    print(proc)
    if test_mode:
        out = processor.run_uproot_job(
            fileset,
            treename="MuonSystem",
            processor_instance=processors[proc],
            executor=processor.futures_executor,
            # executor_args={"schema": BaseSchema, "workers": 10},
            executor_args={"schema": BaseSchema, "workers": 1},
            maxchunks = 1,
            chunksize=1,
        )
    else:
        out = processor.run_uproot_job(
            fileset,
            treename="MuonSystem",
            processor_instance=processors[proc],
            executor=processor.futures_executor,
            executor_args={"schema": BaseSchema, "workers": 10},
            # executor_args={"schema": BaseSchema, "workers": 1},
            # maxchunks = 200,
            # chunksize=10000,
        )
        
        filename = f'hists_cutflows_{proc}_04_17_23.pickle'
        outfile = open(filename, 'wb')
        pickle.dump(out, outfile)
        outfile.close()

Processing 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329/329 [ 0:05:59 < 0:00:00 | 1.3   chunk/s ]
Merging (local) 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329/329 [ 0:06:00 < 0:00:00 | 25.8 merges/s ]

In [19]:
[(thing,out['background'][thing]) for thing in out['background'] if 'numEvents' in thing and '_SF' not in thing]

[('numEvents_pretrigger', 25667521),
 ('numEvents_|muonId| == 13', 25667155),
 ('numEvents_muonHLTReq', 24764401),
 ('numEvents_|muonEta| < 1.5', 22954060),
 ('numEvents_muonPt > 7', 22896397),
 ('numEvents_soft_muon_ID', 22039911),
 ('numEvents_leadMuon_cut', 22039911),
 ('numEvents_posttrigger', 22039911),
 ('numEvents_llp_acc', 22039911),
 ('numEvents_num Cluster > 0', 5032009),
 ('numEvents_dR_gllp_cls < .4', 5032009),
 ('numEvents_dR_lmuon_cls > .8', 3638409),
 ('numEvents_ME1112_veto', 225311),
 ('numEvents_re12_veto', 223242),
 ('numEvents_mb1_veto', 221292),
 ('numEvents_rb1_veto', 220292),
 ('numEvents_muon_veto_pt < 20', 219480),
 ('numEvents_-5 < cls_time < 12.5', 37360),
 ('numEvents_|cls_timeSpread| < 20', 25562),
 ('numEvents_|cls_eta| < 1.9', 4713),
 ('numEvents_cut_based_ID', 2662),
 ('numEvents_cls_size > 130', 97)]